In [1]:
import os
import pandas as pd
from pyswarm import pso
from sklearn.svm import SVR
from math import sqrt                                                                            
import numpy as np
from sklearn.model_selection import KFold, cross_val_score as CVS, train_test_split as TTS
from sklearn.metrics import mean_squared_error as MSE
import pandas as pd


In [2]:
# -*- coding: UTF-8 -*-

Data = pd.read_csv(r"F:\lastpaper\data\L8_20191105_docCDOM_2.csv"
                   ,encoding ='unicode_escape'
                  ,index_col = 0)

X = Data.iloc[:,2:9]

Y = Data.iloc[:,0]
Xtrain,Xtest,Ytrain,Ytest = TTS(X,Y,test_size=0.3,random_state=40)

In [7]:
def svrPso(params):
#    Xtrain, Xtest, ytrain, ytest = TTS(X,Y,test_size = 0.2,random_state = 20)
    kf = KFold(n_splits=fold_count
               ,shuffle=True
               ,random_state = 20
              )
    rmseTotal = 0
    for train, test in kf.split(X,Y):
        
        # Prepare Training and Testing data set. Use cross validation output to split the data sets. 
        #X_train, X_test, y_train, y_test = X.iloc[train,:], X.iloc[test,:], Y[train,0], Y[test,1]
        #previousarrvial = Y[test,1]
        nn = SVR( C=params[0], epsilon = params[1])
        nn = nn.fit(X.iloc[train,:],Y[train])
        result = nn.predict(X.iloc[test,:])
        #thisrmse = sqrt(sum((abs(y_test-result))**2)/len(X_test))
        thisrmse = MSE(Y[test], result)
        rmseTotal = rmseTotal + thisrmse
        
    rmseCV = rmseTotal/fold_count 
    #print('Optimizing the Parameters ..... C = {c}, epsilon={e}, RMSE={m}'.format(c=params[0], e=params[1], m=rmseCV))
    return rmseCV

In [8]:
# svr参数
#     kernel: 函数类型，可选"linear","poly","rbf","sigmoid","precomputed".默认rbf
#     degree: 多项式最高次数，只针对poly，默认为3
#     gamma: 函数系数，默认为"auto"，设置为1/n_features
#     coef0: 常数项，默认为0，只针对poly和sigmoid函数
#     tol: 误差达到指定值则停止训练，默认0.001
#     C: 惩罚参数，c越大，误分惩罚越大，模型测试准确性变高，但是泛化能力弱；C越小则泛化能力变强，默认为1
#     epsilon: 不受误差惩罚的区间宽度，默认0.1
#     shrinking: 支持向量的简单化问题的技术：支持向量只占训练样本的少部分，并且大部分支持向量的拉格朗日乘子等于C，默认为True
#     cache_size: 内核缓存大小，默认200M
#     verbose: 是否启用详细输出，默认为False
#     max_iter: 最大迭代次数，默认为-1，表示无穷次迭代


# pso参数
#     func: 最小化的目标
#     lb: 优化参数的下限
#     ub: 优化参数的上限

#     ieqcons:
#     f_ieqcons:
#     args:
#     kwargs:
#     swarmsize:粒子群的大小，初始为200，改为10，用于调整pso运行次数
#     omega: 粒子速度缩放系数，默认0.5
#     phip: 搜索远离当前层最佳位置的缩放因子，默认0.5
#     phig: 搜索远离当前群体最佳位置的缩放因子，默认0.5
#     maxiter: 粒子群最大迭代次数，默认100
#     minstep: 粒子群最佳位置小于最小步长时，搜索终止，默认1e-8
#     minfunc: 目标参数(MAPE)的最低要求满足时，搜索终止，默认1e-8
#     debug：进度条，默认为False



In [10]:

print("************  Initializing PSO based SVR *****************")    
# Load .csv Data from filestore.
#load_csvdata()

global fold_count  # K-Fold Cross validation.
fold_count = 5
#Define lower bound (lb) and upper bound (ub) for parameter(s) to be optimized 
# To optimize more than one parameter, define lb and up values as lower and upper bound of each parameter = [p1,p2,p3,.....pn]
#  = [C   , epsilon, ....... other parameters]
lb = [10.0, 0.01]
ub = [500.0, 1]


# xopt: array of parameter(s) for optimized values --- fopt: optimized objective function value  --- svrPSO: function to be optimized.
xopt, fopt = pso(svrPso, lb, ub, maxiter=100, debug=True,phip=0.5, omega= 0.9, swarmsize=100, minfunc=0.001 )

print(" ")
print("************ Objective Function optimized *****************")
print(" ")
print('ALL Parameters optimized: C = {c}, epsilon={e}, Overall RMSE={m}'.format(c=xopt[0], e=xopt[1], m=fopt))
print(" ")
print(" ")
print("************  Optimization Finished *****************")

************  Initializing PSO based SVR *****************
No constraints given.
Best after iteration 1: [171.67568055   0.74654199] 0.34208856151582534
Best after iteration 2: [171.67568055   0.74654199] 0.34208856151582534
Best after iteration 3: [171.67568055   0.74654199] 0.34208856151582534
Best after iteration 4: [171.67568055   0.74654199] 0.34208856151582534
Best after iteration 5: [171.67568055   0.74654199] 0.34208856151582534
Best after iteration 6: [171.67568055   0.74654199] 0.34208856151582534
Best after iteration 7: [171.67568055   0.74654199] 0.34208856151582534
New best for swarm at iteration 8: [152.62886873   0.74696723] 0.341922891823285
Stopping search: Swarm best objective change less than 0.001
 
************ Objective Function optimized *****************
 
ALL Parameters optimized: C = 152.62886872618444, epsilon=0.7469672268506768, Overall RMSE=0.341922891823285
 
 
************  Optimization Finished *****************


In [12]:
## 模型封装


import pickle
from sklearn.linear_model import LinearRegression as LR

bst =  SVR(C=152.62886872618444, epsilon = 0.7469672268506768).fit(Xtrain,Ytrain)

#保存模型
pickle.dump(bst, open("SVR_cdom.dat","wb"))
#注意，open中我们往往使用w或者r作为读取的模式，但其实w与r只能用于文本文件，当我们希望导入的不是文本文件，而
#是模型本身的时候，我们使用"wb"和"rb"作为读取的模式。其中wb表示以二进制写入，rb表示以二进制读入
# 导出模型
#loaded_model = pickle.load(open("SVR_cdom.dat", "rb"))

In [14]:
from time import time
import datetime

time0 = time()
svr_model = SVR(C=152.62886872618444, epsilon = 0.7469672268506768).fit(Xtrain, Ytrain)

time= datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f") #计算时间戳并转换为时分秒

ypreds = svr_model.predict(Xtest)
yprods = svr_model.predict(Xtrain)

from sklearn.metrics import mean_squared_error as MSE,r2_score
test_mse=MSE(Ytest,ypreds)
test_r2 = r2_score(Ytest,ypreds)
fit_r2 = r2_score(Ytrain,yprods)
fit_mse = MSE(Ytrain,yprods)
prams= [['Fittime','FitMSE','FitR2','TestMSE','TestR2'],
        [time,fit_mse,fit_r2,test_mse,test_r2]]
pd.DataFrame(prams)

,0,1,2,3,4
0,Fittime,FitMSE,FitR2,TestMSE,TestR2
1,00:00:002992,0.342765,0.675078,0.36046,0.598244


In [16]:
# coding=UTF-8
import xlrd
import xlwt

def write_excel_xls(path, sheet_name, value):
    workbook = xlwt.Workbook()  # 新建一个工作簿
    for k in range(len(value)):
        index = len(value[k])  # 获取需要写入数据的行数
        data = value[k]
        name = sheet_name[k]
        sheet = workbook.add_sheet(name)  # 在工作簿中新建一个表格
        for i in range(0, index):
            for j in range(0, len(data[i])):
                sheet.write(i, j, str(data[i][j]))  # 像表格中写入数据（对应的行和列）
    workbook.save(path)  # 保存工作簿
    print("xls格式表格写入数据成功！")

book_name_xls = 'SVR.xls'
 
sheet_name_xls = ['prams','testdata','traindata']
 
value = [[prams],[Ytest,ypreds],[Ytrain,yprods]]
 
write_excel_xls(book_name_xls,sheet_name_xls,value)

xls格式表格写入数据成功！
